Now we want to load the station metadata available from GIOŚ

In [ ]:
import pandas as pd

df_meta = pd.read_excel("./meta.xlsx", decimal=",")
df_meta.set_index("Nr", inplace=True, drop=True)
df_meta.columns = (
    df_meta.columns
    .str.strip()
    .str.lower()
    .str.replace(r'\s+', '_', regex=True)
)
df_meta = df_meta.rename(columns={
    'stary_kod_stacji_(o_ile_inny_od_aktualnego)': 'stary_kod',
    'wgs84_φ_n': 'lat',
    'wgs84_λ_e': 'lon'
})

In [6]:
df_meta.reset_index(inplace=True)
df_meta.set_index('kod_stacji', inplace=True)
df_meta

,Nr,kod_międzynarodowy,nazwa_stacji,stary_kod,data_uruchomienia,data_zamknięcia,typ_stacji,typ_obszaru,rodzaj_stacji,województwo,miejscowość,adres,lat,lon
kod_stacji,,,,,,,,,,,,,,
DsBialka,1,NaN,Białka,NaN,1990-01-03,2005-12-31,przemysłowa,podmiejski,kontenerowa stacjonarna,DOLNOŚLĄSKIE,Białka,NaN,51.197783,16.117390
DsBielGrot,2,NaN,Bielawa - ul. Grota Roweckiego,NaN,1994-01-02,2003-12-31,tło,miejski,w budynku,DOLNOŚLĄSKIE,Bielawa,ul. Grota Roweckiego 6,50.682510,16.617348
DsBogatFrancMOB,3,PL0602A,Bogatynia Mobil,DsBogatMob,2015-01-01,2015-12-31,tło,miejski,mobilna,DOLNOŚLĄSKIE,Bogatynia,ul. Francuska/Kręta,50.940998,14.916790
DsBogChop,4,PL0315A,Bogatynia - Chopina,NaN,1996-01-01,2013-12-31,przemysłowa,miejski,kontenerowa stacjonarna,DOLNOŚLĄSKIE,Bogatynia,ul. Chopina 35,50.905856,14.967175
DsBogZatonieMob,5,PL0576A,Bogatynia - Mobil,NaN,2012-01-01,2012-12-31,przemysłowa,miejski,mobilna,DOLNOŚLĄSKIE,Bogatynia,"ul. Konrada, Zatonie",50.943245,14.913327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZpSzczPilsud,1112,PL0249A,"Szczecin, ul. Piłsudskiego","ZpSzczecin002, ZpSzczPils02",2004-12-31,NaT,komunikacyjna,miejski,kontenerowa stacjonarna,ZACHODNIOPOMORSKIE,Szczecin,ul. Piłsudskiego 1,53.432169,14.553900
ZpSzczWSSEEnerg,1113,NaN,Energetyków,NaN,1992-01-01,2003-12-31,komunikacyjna,miejski,w budynku,ZACHODNIOPOMORSKIE,Szczecin,ul. Energetyków 2,53.420475,14.561934
ZpSzczWSSESped6,1114,NaN,Spedytorska,NaN,1992-01-01,2004-01-01,tło,miejski,kontenerowa stacjonarna,ZACHODNIOPOMORSKIE,Szczecin,ul. Spedytorska 6,53.415043,14.555347


In [10]:

df_stacje = pd.read_csv("stations_complete_2016_2019.csv", index_col=0, parse_dates=True)

stacje_w_csv = df_stacje.columns

df_meta = df_meta.loc[df_meta.index.intersection(stacje_w_csv), ["lat", "lon"]]

In [8]:
import osmnx as ox
print(ox.__version__)

2.0.6


Now we want to add some additional informations to station metadata;

We will use open street map to extract some important features:

In [ ]:
import geopandas as gpd
import pandas as pd
import osmnx as ox
from shapely.geometry import Point
import numpy as np

# konfiguracja
ox.settings.use_cache = True
ox.settings.log_console = False

buffer_radius = 5000  # m

def compute_features(lat, lon, buffer_radius=2000):
    try:
        pt = Point(lon, lat)  # shapely: (lon, lat)
        buffer_poly = ox.utils_geo.buffer_geometry(pt, buffer_radius)

        buffer_3857 = gpd.GeoSeries([buffer_poly], crs="EPSG:4326").to_crs(epsg=3857).iloc[0]
        pt_3857 = gpd.GeoSeries([pt], crs="EPSG:4326").to_crs(epsg=3857).iloc[0]

        # === DROGI ===
        try:
            G = ox.graph_from_point((lat, lon), dist=buffer_radius, network_type="drive")
            _, edges = ox.graph_to_gdfs(G)
            edges = edges.to_crs(epsg=3857)

            edges_in = edges[edges.geometry.intersects(buffer_3857)]
            total_road_length_km = edges_in.length.sum() / 1000 if not edges_in.empty else np.nan
            nearest_road_dist_m = edges_in.distance(pt_3857).min() if not edges_in.empty else np.nan

        except Exception as e:
            print(f"[Błąd dróg] ({lat}, {lon}): {e}")
            total_road_length_km = np.nan
            nearest_road_dist_m = np.nan

        try:
            buildings = ox.features_from_polygon(buffer_poly, tags={"building": True})
            buildings = buildings[buildings.geometry.geom_type.isin(["Polygon", "MultiPolygon"])]

            buildings = buildings.to_crs(epsg=3857)
            buildings_in = buildings[buildings.geometry.intersects(buffer_3857)]
            building_count = len(buildings_in) if not buildings_in.empty else 0

        except Exception as e:
            print(f"[Błąd budynków] ({lat}, {lon}): {e}")
            building_count = np.nan

        return nearest_road_dist_m, total_road_length_km, building_count

    except Exception as e:
        print(f"[Błąd ogólny] ({lat}, {lon}): {e}")
        return np.nan, np.nan, np.nan


nearest_road_dist_list = []
total_road_length_list = []
building_count_list = []
x = 1
for i, row in df_meta.iterrows():
    nr, trl, bc = compute_features(row.lat, row.lon, buffer_radius)
    nearest_road_dist_list.append(nr)
    total_road_length_list.append(trl)
    building_count_list.append(bc)
    print(f"Przetworzono {x}/{len(df_meta)} lokalizacji")
    x+=1

df_meta["nearest_road_dist_m"] = nearest_road_dist_list
df_meta["total_road_length_km"] = total_road_length_list
df_meta["building_count"] = building_count_list

print(df_meta)


Przetworzono 1/61 lokalizacji
Przetworzono 2/61 lokalizacji
Przetworzono 3/61 lokalizacji
Przetworzono 4/61 lokalizacji
Przetworzono 5/61 lokalizacji
Przetworzono 6/61 lokalizacji
Przetworzono 7/61 lokalizacji
Przetworzono 8/61 lokalizacji
Przetworzono 9/61 lokalizacji
Przetworzono 10/61 lokalizacji
Przetworzono 11/61 lokalizacji
Przetworzono 12/61 lokalizacji
Przetworzono 13/61 lokalizacji
Przetworzono 14/61 lokalizacji
Przetworzono 15/61 lokalizacji
Przetworzono 16/61 lokalizacji
Przetworzono 17/61 lokalizacji
Przetworzono 18/61 lokalizacji
Przetworzono 19/61 lokalizacji
Przetworzono 20/61 lokalizacji
Przetworzono 21/61 lokalizacji
[Błąd budynków] (49.619281, 20.714403): HTTPSConnectionPool(host='overpass-api.de', port=443): Max retries exceeded with url: /api/interpreter (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000160B83B36D0>, 'Connection to overpass-api.de timed out. (connect timeout=180)'))
Przetworzono 22/61 lokalizacji
Przetworzono 23/61 

In [ ]:
'''
import requests

try:
    r = requests.get("https://overpass-api.de/api/interpreter")
    print(r.status_code)
except Exception as e:
    print("Błąd sieci:", e)
'''

400


In [13]:

from IPython.display import display, HTML

# przykład dla DataFrame
display(HTML(f"<div style='max-height:300px; overflow:auto;'>{df_meta.to_html()}</div>"))

df_meta
df_meta.to_csv('metadata5000.csv')

,lat,lon,nearest_road_dist_m,total_road_length_km,building_count
DsLegAlRzecz,51.204503,16.180513,98.613062,942.949585,15745.0
DsOsieczow21,51.317630,15.431719,40.258760,148.954010,2334.0
DsWalbrzWyso,50.768729,16.269677,170.100903,817.813995,15897.0
DsWrocNaGrob,51.103456,17.059225,97.176639,1518.020860,60249.0
DsZgorBohGet,51.150391,15.008175,138.987771,988.524194,28363.0
KpBydFieldor,53.151452,18.132062,22.380118,711.014879,8995.0
KpGrudSienki,53.491831,18.752503,108.389527,765.180131,22104.0
KpToruDziewu,53.028647,18.666103,97.101206,1339.475703,21483.0
KpZielBoryTu,53.662117,17.934017,155.819124,123.469175,665.0
LbBiaPodOrze,52.029194,23.149389,28.987375,888.236469,16920.0


We perform this one more time to add locations with errors

In [3]:
import geopandas as gpd
import pandas as pd
import osmnx as ox
from shapely.geometry import Point
import numpy as np

# konfiguracja
ox.settings.use_cache = True
ox.settings.log_console = False

buffer_radius = 5000  # m

def compute_features(lat, lon, buffer_radius=2000):
    try:
        # Tworzymy punkt i bufor (buffer_geometry automatycznie konwertuje do metrycznego CRS)
        pt = Point(lon, lat)  # shapely: (lon, lat)
        buffer_poly = ox.utils_geo.buffer_geometry(pt, buffer_radius)

        # Konwersje do EPSG:3857 tylko raz
        buffer_3857 = gpd.GeoSeries([buffer_poly], crs="EPSG:4326").to_crs(epsg=3857).iloc[0]
        pt_3857 = gpd.GeoSeries([pt], crs="EPSG:4326").to_crs(epsg=3857).iloc[0]

        # === DROGI ===
        try:
            G = ox.graph_from_point((lat, lon), dist=buffer_radius, network_type="drive")
            _, edges = ox.graph_to_gdfs(G)
            edges = edges.to_crs(epsg=3857)

            edges_in = edges[edges.geometry.intersects(buffer_3857)]
            total_road_length_km = edges_in.length.sum() / 1000 if not edges_in.empty else np.nan
            nearest_road_dist_m = edges_in.distance(pt_3857).min() if not edges_in.empty else np.nan

        except Exception as e:
            print(f"[Błąd dróg] ({lat}, {lon}): {e}")
            total_road_length_km = np.nan
            nearest_road_dist_m = np.nan

        # === BUDYNKI ===
        try:
            # Uwaga: 'True' w OSMnx filtruje wszystkie budynki, ale w starych wersjach trzeba podać listę wartości
            buildings = ox.features_from_polygon(buffer_poly, tags={"building": True})
            buildings = buildings[buildings.geometry.geom_type.isin(["Polygon", "MultiPolygon"])]

            buildings = buildings.to_crs(epsg=3857)
            buildings_in = buildings[buildings.geometry.intersects(buffer_3857)]
            building_count = len(buildings_in) if not buildings_in.empty else 0

        except Exception as e:
            print(f"[Błąd budynków] ({lat}, {lon}): {e}")
            building_count = np.nan

        return nearest_road_dist_m, total_road_length_km, building_count

    except Exception as e:
        print(f"[Błąd ogólny] ({lat}, {lon}): {e}")
        return np.nan, np.nan, np.nan


df_meta = pd.read_csv("metadata5000.csv")
# Filtrujemy tylko wiersze, w których przynajmniej jedna z kolumn wynikowych ma NaN
rows_to_process = df_meta[
    df_meta[["nearest_road_dist_m", "total_road_length_km", "building_count"]].isna().any(axis=1)
]

x = 1
for i, row in rows_to_process.iterrows():
    nr, trl, bc = compute_features(row.lat, row.lon, buffer_radius)
    df_meta.at[i, "nearest_road_dist_m"] = nr
    df_meta.at[i, "total_road_length_km"] = trl
    df_meta.at[i, "building_count"] = bc
    print(f"Przetworzono {x}/{len(rows_to_process)} lokalizacji")
    x += 1


Przetworzono 1/7 lokalizacji
Przetworzono 2/7 lokalizacji
Przetworzono 3/7 lokalizacji
Przetworzono 4/7 lokalizacji
Przetworzono 5/7 lokalizacji
Przetworzono 6/7 lokalizacji
Przetworzono 7/7 lokalizacji


In [4]:
from IPython.display import display, HTML

# przykład dla DataFrame
display(HTML(f"<div style='max-height:300px; overflow:auto;'>{df_meta.to_html()}</div>"))

df_meta
df_meta.to_csv('metadata5000best.csv')

,Unnamed: 0,lat,lon,nearest_road_dist_m,total_road_length_km,building_count
0,DsLegAlRzecz,51.204503,16.180513,98.613062,942.949585,15745.0
1,DsOsieczow21,51.317630,15.431719,40.258760,148.954010,2334.0
2,DsWalbrzWyso,50.768729,16.269677,170.100903,817.813995,15897.0
3,DsWrocNaGrob,51.103456,17.059225,97.176639,1518.020860,60249.0
4,DsZgorBohGet,51.150391,15.008175,138.987771,988.524194,28363.0
5,KpBydFieldor,53.151452,18.132062,22.380118,711.014879,8995.0
6,KpGrudSienki,53.491831,18.752503,108.389527,765.180131,22104.0
7,KpToruDziewu,53.028647,18.666103,97.101206,1339.475703,21483.0
8,KpZielBoryTu,53.662117,17.934017,155.819124,123.469175,665.0
9,LbBiaPodOrze,52.029194,23.149389,28.987375,888.236469,16920.0
